# IEEE 802.11n

A etapa de investigação do IEEE 802.11n será dividida em duas etapas, a primeira operando na frequência de 2.4 GHz e a seguinte operando na frequência de 5 GHz

## Requisitos:
* Copiar o exemplo __rate-adaptation-distance.cc__, encontrado em __ns-3.29/examples/wireless__, para a pasta __ns-3.29/scratch__;
* Instalar o gnuplot.

## Alterações:
Para investigar os efeitos dos algoritmos de adaptação de taxa nos diversos padrões IEEE 802.11 serão realizadas algumas alterações no exemplo __rate-adaptation-distance.cc__. A imagem a seguir exibe os principais parâmetros do script que podem ser alterados, resultando em diferentes cenários de análise. 

![parametros](./figs/ratedistance.png)

* __staManager__: Define o algoritmo de adaptação utilizado pelo nó STA;
* __apManager__: Define o algoritmo de adaptação utilizado pelo nó AP;
* __standard__: Define qual padrão IEEE 802.11 será utilizado na simulação;
* __OutputFileName__: Define o nome do arquivo de saída da simulação;
* __shortGuardInterval__: Habilita/Desabilita o uso de um Intervalo curto de guarda;
* __chWidth__: Define a largura do canal utilizado na simulação;
* __ap1_x/y__: Define a posição inicial do nó AP;
* __sta1_x/y__: Define a posição inicial do nó STA;
* __steps__: Define a quantidade de passos tomados;
* __stepsSize__: Define a distância tomada em cada passo;
* __stepsTime__: Define o intervalo entre passos.

Neste tutorial, as alterações serão focadas nos algoritmos de adaptação e padrões utilizados. Ao final da simulação é gerado um arquivo plt, usado para plotar o gráfico de throughput vs distância por meio da ferramenta gnuplot.

É importante salientar que o ns-3, através do módulo wifi, disponibiliza diversos algoritmos de adaptação para construir cenários de simulação variados. Nem todos os algoritmos de adaptação funcionam com qualquer padrão 802.11, assim ficou decidida a utilização do algoritmo de adaptação minstrel. Ao final deste tutorial será possível facilmente testar o funcionamento dos outros algoritmos disponíveis no simulador, encontrados no diretório __~/ns-3.29/src/wifi/model__ .

### IEEE 802.11n @ 2.4 GHz

O cenário de simulação para este padrão foi configurado do seguinte modo:

![conf2.4](./FIGS/conf24.png)

Perceba que foi utilizado o algoritmo de adaptação __minstrelHt__ ao invés do __minstrel__. Isso acontece devido ao algoritmo minstrel padrão não suportar as altas taxas de transferências (High Throughput - HT) presentes no 802.11n. Seguindo os passos realizados nas simulações anteriores, obtemos o gráfico de throughput vs distância para o IEEE 802.11n operando na frequência de 2.4 GHz.

![throughput2.4](./FIGS/throughput2_4_n.png)

### IEEE 802.11n @ 5 GHz

Configuramos o cenário de simulação para este padrão do seguinte modo:

![conf5](./FIGS/conf5.png)

Seguindo os passos realizados nas simulações anteriores, obtemos o gráfico de throughput vs distância para o IEEE 802.11n operando na frequência de 5 GHz.

![throughput5](./FIGS/throughput5_n.png)